In [80]:
def DBConnection(DBstatus):
    conn = cx_Oracle.connect("TRSADMIN", "TRS", "DBX03T")
    if DBstatus == True:
        return conn
    else:
        conn.close()

In [101]:
def createFordAcknowledgement(rows):
    import shutil
    date = datetime.datetime.now()
    date = date.strftime('%c')

    logFile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "a")
    
    DBstatus = True
    conn = DBConnection(DBstatus)
    c = conn.cursor()
    
    c.execute('select FORD_ACKNOWLEDGEMENT_BATCH_SEQ.nextval from DUAL')
    for result in c:
        batchNum = result[0]
        if batchNum == '':
            date = datetime.datetime.now()
            date = date.strftime('%c')
            print(f"{date} **WARNING** - Next FORD_ACKNOWLEDGEMENT_BATCH_SEQ number not returned from Oracle", file = logFile)
            
    # Create and write the ack file")
    fileName = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Book Notes 10/DOT_{batchNum}_ack.txt"
    infile = open(fileName, "w")
    print(f"{batchNum} {rows}-records processed", file = infile)
    infile.close()
    # Make a copy in the archive folder
    archiveName = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Chapter 11/{date[0:11]}DOT_{batchNum}_ack.txt"
    shutil.copy(fileName, archiveName)

In [164]:
createFordAcknowledgement(50) #'select FORD_ACKNOWLEDGEMENT_BATCH_SEQ.nextval from DUAL'

In [85]:
def executeStoredProcedure(sourceID, record, firstRecord):
    import cx_Oracle
    
    DBstatus = True
    conn = DBConnection(DBstatus)
    c = conn.cursor()
    
    c.callproc('maintain_tire_registration',[sourceID, record, firstRecord] ) # use triple quotes if you want to spread your query across multiple lines
    
    #print(f"First: {sourceID}, record: {record}, and firstRecord: {firstRecord}")

In [160]:
def getTireRegistrationRecords(fileName, sourceID):
    # Need to check length of each record not how many records; include for loop
    record_count = 0
    try:
        """Tries to open the file and go through its contents; if the file is over 375 records an error is logged"""
        print("YES")
        firstRecord = "True"
        infile = open(fileName, "r")
        rows = infile.readlines()
        rows_string = ''.join(rows)
        rows_list = rows_string.split('\n')
        #print(f"Rows_list{rows_list}")
        logFile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "a")
        for record in rows_list:
            length = len(record)
            print(f"Record: {record}")
            date = datetime.datetime.now()
            date = date.strftime('%c')
            record_count += 1
        
            if length > 375:
                print(f"{date} **Warning** - Record too long - Record Count {length} record: {record}", file = logFile)
            else:
                #executeStoredProcedure(sourceID, record, firstRecord)
                firstRecord = 'False'
                print('yep')
                
        infile.close()
        print(f"{date} INFO - Processed - Records: {record_count}  SourceID: {sourceID}", file = logFile)
        logFile.close()
        
        
        """For special cases when SourceID is from Ford"""
        if sourceID == 'FORD':
            createFordAcknowledgement(record_count)
        else: 
            finished = 'yepp'

    except:
        print('NOO')
        date = datetime.datetime.now()
        date = date.strftime('%c')
        print(f"{date} **ERROR** - Parsing {fileName} ERROR - Record Count {record_count} File Name: {fileName}", file = logFile)
        logFile.close()
      

In [161]:
getTireRegistrationRecords('C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/tirereg_canada.txt', 'CANADA')

YES
Record: 001000090E667GOODYEAR CANADA_blessing INC.          450 KIPLING AVENUE       ETOBICOKE      ONM8Z 5E1  120319                 ROBERTSON                     31 ASTERWIND CRES        BRAMPTON       ONL6R1V8   G004PDAVKH0019                                                                                                  011299010004C   R
yep
Record: 001000090E668GOODYEAR CANADA_blessing INC.          450 KIPLING AVENUE       ETOBICOKE      ONM8Z 5E1  120319                 ROBERTSON                     31 ASTERWIND CRES        BRAMPTON       ONL6R1V8   G004PDAVKH0019                                                                                                  011299010004C   R
yep


In [162]:
import datetime
import os.path
from os import path
import shutil

def main():
    iniFirst = []
    iniSecond = []
    trFilesFirst = []
    trFilesSecond = []
    DBstatus = False
    
    """Try and find the file, if the file is found logic is applied to its contents and saved as variables
    Otherwise an error is written to the log file"""
    
    try:
        infile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Book Notes 10/TireRegistration.ini", "r")
        
        '''Loops through the file splitting each line by a '='; then saves the split in first & second
        then appends first and second to iniFirst and iniSecond respectivley'''
        
        for line in infile:
            first_infile, second_infile = line.split('=')
            #print(f"First:{first} Second:{second}")
            iniFirst.append(first_infile), iniSecond.append(second_infile)
        
        '''Goes through the iniFirst list and verifies the neccessary value, then saves a variable with
        the value of the corresponding iniSecond value'''
        
        if iniFirst[0] == 'Database':
            dbName = iniSecond[0]
            
        if iniFirst[1] == 'InputDir':
            inputDir = iniSecond[1]
            
        if iniFirst[2] == 'OutputDir':
            outputDir = iniSecond[2]
            
        if iniFirst[3] == 'ArchiveDir':
            archiveDir = iniSecond[3]
            
        if iniFirst[4] == 'LogDir':
            logDir = iniSecond[4]
            
        if iniFirst[5] == 'FileCount':
            fileCount = iniSecond[5]
            
        if iniFirst[6] == 'InputFiles':
            inputFiles = iniSecond[6]
            trFiles = inputFiles.split(",")
        
    
    except:
        
        """If the file is not found write to the log file the error and the date/time"""
        
        date = datetime.datetime.now()
        date = date.strftime('%c')
        outfile = open('ts_logfile.txt', 'w')
        print(f"{date} **Error** - Could not find C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Book Notes 10/TireRegistration.ini", file = outfile)

    finally:
        print('Done')
        
    # Closes the file
    infile.close()
    
    
    '''Trys to append to the log file. If no log file is found it creates one. It then goes through the trFiles
    and splits the contents by a ':' '''
    
    try:
        infile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "a")
        print('Sure')
    except:
        infile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "w")
        print('no')
    finally:
        for record in trFiles:
            first_trFiles, second_trFiles = record.split(":")
            trFilesFirst.append(first_trFiles), trFilesSecond.append(second_trFiles)
    
        '''Writes to the log file if it find the file otherwise writes that it couldnt find the value'''
        date = datetime.datetime.now()
        date = date.strftime('%c')
        
        date2 = date
        date2 = date2[:10] + date2[19:] # Making the date ok for placement of a filename (Removing "/")
        counter = 0
        
        for file in trFilesSecond:
            # Would be inputDir in production
            fileName = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/{trFilesSecond[0+counter]}" # This would be the inputDir file
            if path.exists(fileName):
                print(f"{date} INFO - {inputDir} {trFilesSecond[0+counter]} {trFilesFirst[0+counter]} file found and opening processing", file = infile)
                getTireRegistrationRecords(fileName, trFilesFirst[0+counter])
                DBConnection(DBstatus=False)
                #fileName = inputDir + trFilesSecond[0+counter] # In Prod this is what it would be
                newFile = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Chapter 6/{trFilesSecond[0+counter]}" # This would be the archiveDir
                shutil.move(fileName, newFile) # Move the file to archive
                newFile2 = newFile[:-4] # Get's rid of .txt so I can append the date
                os.rename(newFile, newFile2+date2+'.txt') # Renames the file with the date appended
            else:
                print(f"{date} INFO - {inputDir} {trFilesSecond[0+counter]} {trFilesFirst[0+counter]} file NOT found", file = infile)
            counter += 1
            
        #print(trFilesFirst, trFilesSecond)
        infile.close()


In [163]:
main()

Done
Sure
YES
Record: 001000090E667GOODYEAR CANADA_blessing INC.          450 KIPLING AVENUE       ETOBICOKE      ONM8Z 5E1  120319                 ROBERTSON                     31 ASTERWIND CRES        BRAMPTON       ONL6R1V8   G004PDAVKH0019                                                                                                  011299010004C   R
yep
Record: 001000090E668GOODYEAR CANADA_blessing INC.          450 KIPLING AVENUE       ETOBICOKE      ONM8Z 5E1  120319                 ROBERTSON                     31 ASTERWIND CRES        BRAMPTON       ONL6R1V8   G004PDAVKH0019                                                                                                  011299010004C   R
yep
YES
Record: 05790,"Blessing P. Blessing Inc","11442 S. LONE PEAK",DRAPER,UT,84020,011420,"OKLAND CONSTRUCTION COMPANY INC ","1978 S WEST TEMPLE",SLC,UT,84115,1,PL8TJA2R2819
yep
Record: 05791,"Blessing P. Blessing Inc","11442 S. LONE PEAK",DRAPER,UT,84020,011420,"OKLAND CONSTRUCTION COMPA

In [9]:
new = infile.readlines()

In [10]:
new

['Database=DBX03T\n',
 'InputDir=D:\\TireRegistration\\Input\\\n',
 'OutputDir=D:\\TireRegistration\\Output\\\n',
 'ArchiveDir=D:\\TireRegistration\\Archive\\\n',
 'LogDir=D:\\TireRegistration\\Log\\\n',
 'FileCount=11\n',
 'InputFiles=CONRADS:tirereg_conrads.txt,CANADA:tirereg_canada.txt,CASNET:tirereg_casnet.txt,CIMS:tirereg_cims.txt,RETAIL:tirereg_retail.txt,TRR:tirereg_trr.txt,WINGFOOT:tirereg_wingfoot.txt,FORD:tirereg_ford.txt,VOL:tirereg_vol.txt,TIREMTRX:Goodyear.csv,FCA:tirereg_fca.txt']

In [189]:
import os.path
from os import path
path.exists("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/tirereg_conrads.txt")

True

In [1]:
import cx_Oracle

con = cx_Oracle.connect('TRSADMIN/TRS@10.128.252.205/orcl')
print(con.version())

con.close()

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://oracle.github.io/odpi/doc/installation.html#windows for help

In [23]:
pip install cx_Oracle --upgrade


Note: you may need to restart the kernel to use updated packages.


In [3]:
import cx_Oracle

dsn_tns = cx_Oracle.makedsn('ahqttd1', '1521', service_name='TRS01T.world') # if needed, place an 'r' before any parameter in order to address special characters such as '\'.
conn = cx_Oracle.connect(user=r'TRSADMIN', password='TRS', dsn=dsn_tns) # if needed, place an 'r' before any parameter in order to address special characters such as '\'. For example, if your user name contains '\', you'll need to place 'r' before the user name: user=r'User Name'
c = conn.cursor()
c.execute('select * from INTERFACE_BATCHES') # use triple quotes if you want to spread your query across multiple lines
for row in c:
    print (row[0], '-', row[1]) # this only shows the first two columns. To add an additional column you'll need to add , '-', row[2], etc.
#conn.close()

DatabaseError: DPI-1047: Cannot locate a 64-bit Oracle Client library: "The specified module could not be found". See https://oracle.github.io/odpi/doc/installation.html#windows for help

In [ ]:
# Example Connection
conn = cx_Oracle.connect("TRSADMIN", "TRS", "DBX03T")
c = conn.cursor()
c.callproc('maintain_tire_registration',[pSourceID,TireRegRecord,FirstRecord] ) # use triple quotes if you want to spread your query across multiple lines

In [2]:
# Will use this within the execute stored procedure function
import cx_Oracle

conn = cx_Oracle.connect("TRSADMIN", "TRS", "DBX03T")
c = conn.cursor()
c.callproc('maintain_tire_registration',['CANADA',"001000090E667GOODYEAR CANADA INC.          450 KIPLING AVENUE       ETOBICOKE      ONM8Z 5E1  030420                 ROBERTSON                     31 ASTERWIND CRES        BRAMPTON       ONL6R1V8   G004PDAVKH0019" ,"True"] ) # use triple quotes if you want to spread your query across multiple lines


['CANADA',
 '001000090E667GOODYEAR CANADA INC.          450 KIPLING AVENUE       ETOBICOKE      ONM8Z 5E1  030420                 ROBERTSON                     31 ASTERWIND CRES        BRAMPTON       ONL6R1V8   G004PDAVKH0019',
 'True']

In [103]:
import datetime
import os.path
from os import path
import shutil

def DBConnection(DBstatus):
    conn = cx_Oracle.connect("TRSADMIN", "TRS", "DBX03T")
    if DBstatus == True:
        return conn
    else:
        conn.close()
        
def createFordAcknowledgement(rows = 50):
    import shutil
    date = datetime.datetime.now()
    date = date.strftime('%c')

    logFile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "a")
    
    DBstatus = True
    conn = DBConnection(DBstatus)
    c = conn.cursor()
    
    c.execute('select FORD_ACKNOWLEDGEMENT_BATCH_SEQ.nextval from DUAL')
    for result in c:
        batchNum = result[0]
        if batchNum == '':
            date = datetime.datetime.now()
            date = date.strftime('%c')
            print(f"{date} **WARNING** - Next FORD_ACKNOWLEDGEMENT_BATCH_SEQ number not returned from Oracle", file = logFile)
            
    # Create and write the ack file")
    fileName = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Book Notes 10/DOT_{batchNum}_ack.txt"
    infile = open(fileName, "w")
    print(f"{batchNum} {rows}-records processed", file = infile)
    infile.close()
    # Make a copy in the archive folder
    archiveName = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Chapter 11/{date[0:11]}DOT_{batchNum}_ack.txt"
    shutil.copy(fileName, archiveName)

def executeStoredProcedure(sourceID, record, firstRecord):
    import cx_Oracle
    
    DBstatus = True
    conn = DBConnection(DBstatus)
    c = conn.cursor()
    
    c.callproc('maintain_tire_registration',[sourceID, record,firstRecord] ) # use triple quotes if you want to spread your query across multiple lines
    
    #print(f"First: {sourceID}, record: {record}, and firstRecord: {firstRecord}")

def getTireRegistrationRecords(fileName, sourceID):
    # Need to check length of each record not how many records; include for loop
    try:
        """Tries to open the file and go through its contents; if the file is over 375 records an error is logged"""
        print("YES")
        firstRecord = "True"
        infile = open(fileName, "r")
        rows = infile.readlines()
        rows_string = ''.join(rows)
        rows_list = rows_string.split('\n')
        rows_list.pop()
        #print(f"Rows_list{rows_list}")
        logFile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "a")
        
        for record in rows_list:
            length = len(record)
            print(f"Record: {record}")
            date = datetime.datetime.now()
            date = date.strftime('%c')

            if length > 375:
                print(f"{date} **Warning** - Record too long - Record Count {length} record: {record}", file = logFile)
            else:
                executeStoredProcedure(sourceID, record, firstRecord)
                firstRecord = 'False'
                print('yep')

        infile.close()
        print(f"{date} INFO - Processed - Records: {rows}  SourceID: {sourceID}", file = logFile)
        logFile.close()
        
        """For special cases when SourceID is from Ford"""
        if sourceID == 'FORD':
            createFordAcknowledgement(rows)
        else: 
            finished = 'yepp'

    except:
        print('NOO')
        date = datetime.datetime.now()
        date = date.strftime('%c')
        print(f"{date} **ERROR** - Parsing {fileName} ERROR - Record Count {rows} File Name: {fileName}", file = logFile)
        logFile.close()
        

def main():
    iniFirst = []
    iniSecond = []
    trFilesFirst = []
    trFilesSecond = []
    DBstatus = False
    
    """Try and find the file, if the file is found logic is applied to its contents and saved as variables
    Otherwise an error is written to the log file"""
    
    try:
        infile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Book Notes 10/TireRegistration.ini", "r")
        
        '''Loops through the file splitting each line by a '='; then saves the split in first & second
        then appends first and second to iniFirst and iniSecond respectivley'''
        
        for line in infile:
            first, second = line.split('=')
            #print(f"First:{first} Second:{second}")
            iniFirst.append(first), iniSecond.append(second)
        
        '''Goes through the iniFirst list and verifies the neccessary value, then saves a variable with
        the value of the corresponding iniSecond value'''
        
        if iniFirst[0] == 'Database':
            dbName = iniSecond[0]
            
        if iniFirst[1] == 'InputDir':
            inputDir = iniSecond[1]
            
        if iniFirst[2] == 'OutputDir':
            outputDir = iniSecond[2]
            
        if iniFirst[3] == 'ArchiveDir':
            archiveDir = iniSecond[3]
            
        if iniFirst[4] == 'LogDir':
            logDir = iniSecond[4]
            
        if iniFirst[5] == 'FileCount':
            fileCount = iniSecond[5]
            
        if iniFirst[6] == 'InputFiles':
            inputFiles = iniSecond[6]
            trFiles = inputFiles.split(",")
        
    
    except:
        
        """If the file is not found write to the log file the error and the date/time"""
        
        date = datetime.datetime.now()
        date = date.strftime('%c')
        outfile = open('ts_logfile.txt', 'w')
        print(f"{date} **Error** - Could not find C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Book Notes 10/TireRegistration.ini", file = outfile)

    finally:
        print('Done')
        
    # Closes the file
    infile.close()
    
    
    '''Trys to append to the log file. If no log file is found it creates one. It then goes through the trFiles
    and splits the contents by a ':' '''
    
    try:
        infile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "a")
        print('Sure')
    except:
        infile = open("C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/ts_logfile.txt", "w")
        print('no')
    finally:
        for record in trFiles:
            first, second = record.split(":")
            trFilesFirst.append(first), trFilesSecond.append(second)
    
        '''Writes to the log file if it find the file otherwise writes that it couldnt find the value'''
        date = datetime.datetime.now()
        date = date.strftime('%c')
        
        date2 = date
        date2 = date2[:10] + date2[19:] # Making the date ok for placement of a filename (Removing "/")
        counter = 0
        
        for file in trFilesSecond:
            # Would be inputDir in production
            fileName = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/{trFilesSecond[0+counter]}" # This would be the inputDir file
            if path.exists(fileName):
                print(f"{date} INFO - {inputDir} {trFilesSecond[0+counter]} {trFilesFirst[0+counter]} file found and opening processing", file = infile)
                getTireRegistrationRecords(fileName, trFilesFirst[0+counter])
                DBConnection(DBstatus=False)
                #fileName = inputDir + trFilesSecond[0+counter] # In Prod this is what it would be
                newFile = f"C:/Users/ac01658/OneDrive - Goodyear/Python/Book Notes/Chapter 6/{trFilesSecond[0+counter]}" # This would be the archiveDir
                shutil.move(fileName, newFile) # Move the file to archive
                newFile2 = newFile[:-4] # Get's rid of .txt so I can append the date
                os.rename(newFile, newFile2+date2+'.txt') # Renames the file with the date appended
            else:
                print(f"{date} INFO - {inputDir} {trFilesSecond[0+counter]} {trFilesFirst[0+counter]} file NOT found", file = infile)
            counter += 1
            
        #print(trFilesFirst, trFilesSecond)
        infile.close()


In [ ]:
main()